# Lost in the middle: 长上下文精度问题

当相关联词数量过多时，LLM 的精度会下降。
相关信息在头尾性能最高，所以检索->排序->使用

In [4]:
from langchain_classic.chains.combine_documents.stuff import StuffDocumentsChain, create_stuff_documents_chain
from langchain_classic.chains.llm import LLMChain
from langchain_community.document_transformers import LongContextReorder
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.document_transformers import LongContextReorder

# 创建嵌入模型
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

# 示例文本
text = [
    "篮球是一项伟大的运动。",
    "带我飞往月球是我最喜欢的歌曲之一。",
    "我喜欢机器学习和人工智能技术。",
    "拉里.伯德是一位标志性的NBA球员。"
]

# 创建向量数据库
retrieval = Chroma.from_texts(text, embeddings).as_retriever(
    search_kwargs={"k": 10}
)

query = "关于我的喜好都知道什么?"

# 发现只有部分文档被检索到
docs = retrieval.invoke(query)
print("查询结果数量:", len(docs))
for i, doc in enumerate(docs):
    print(f"文档 {i+1}: {doc.page_content}")

#对检索结果进行重新排序，根据论文的方案
#问题相关性越低的内容块放在中间
#问题相关性越高的内容块放在头尾
reordering = LongContextReorder()
reo_docs = reordering.transform_documents(docs)
print("重新排序后的查询结果数量:", len(reo_docs))
for i, doc in enumerate(reo_docs):
    print(f"文档 {i+1}: {doc.page_content}")


查询结果数量: 10
文档 1: 凯尔特人队是我最喜欢的球队。
文档 2: 我非常喜欢去看电影。
文档 3: 我喜欢机器学习和人工智能技术。
文档 4: 我喜欢机器学习和人工智能技术。
文档 5: 我喜欢机器学习和人工智能技术。
文档 6: 篮球是一项伟大的运动。
文档 7: 篮球是一项伟大的运动。
文档 8: 篮球是一项伟大的运动。
文档 9: 篮球是一项伟大的运动。
文档 10: 带我飞往月球是我最喜欢的歌曲之一。
重新排序后的查询结果数量: 10
文档 1: 我非常喜欢去看电影。
文档 2: 我喜欢机器学习和人工智能技术。
文档 3: 篮球是一项伟大的运动。
文档 4: 篮球是一项伟大的运动。
文档 5: 带我飞往月球是我最喜欢的歌曲之一。
文档 6: 篮球是一项伟大的运动。
文档 7: 篮球是一项伟大的运动。
文档 8: 我喜欢机器学习和人工智能技术。
文档 9: 我喜欢机器学习和人工智能技术。
文档 10: 凯尔特人队是我最喜欢的球队。


## 检测一下精度

In [8]:
from dotenv import load_dotenv
import os
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
load_dotenv()

llm = ChatOpenAI(
    model="gpt-3.5-turbo:free",
    api_key=os.getenv("OPENAI_API_KEY"),
    base_url=os.getenv("OPENAI_BASE_URL"),
    temperature=0,
)

# input是文档内容，template是模板
document_prompt = PromptTemplate(
    input_variables=["page_content"],
    template="{page_content}",
)

stuff_prompt_override ="""Given this text extracts:
----------------------------------------
{context}
----------------------------------------
Please answer the following questions:
{query}
"""

prompt = PromptTemplate(
    template=stuff_prompt_override,
    input_variables=["context","query"]
)

# 创建文档链
stuff_documents_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
    document_prompt=document_prompt
)

# 调用链
result = stuff_documents_chain.invoke({
    "context": reo_docs,  # 使用"context"作为键，而不是"input_documents"
    "query": "我最喜欢做什么事情？"
})

print(result)


你最喜欢去看电影。
